# **Project 2 - Image Processing**
Huy G. Tong $^{1 *}$ \
$^{1}$ *Faculty of Information Technology, VNUHCM - University of Science, Vietnam*


### **ABSTRACT**
Ever since the invention of the first camera in the 1800s, photographs have become an inseparable part of our lives. We took pictures to tell stories of our society, our way of life. Not only that, we used those images to study phenomenons around us: animals, geography, from small things like atoms and microbes to celestial bodies in the depth of space far from our reach. It is not common to find a field of science where photography is not present. As technology for capturing light progressed, new methods of processing images were also created. From developing photographs in darkrooms, now we are able of performing intricate operations on individual pixels of digital images with the help of softwares like Adobe Lighroom or Photoshop. In this project, I will study some basic and commonly used image processing techniques to grasp a deeper understanding of how digital images are manipulated in those programs.

### **IMPLEMENTATION**
This section will display the implementation of some basic image processing functions in Python. First we have to import all necessary packages:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import PIL
import time

#### Brightness adjustment

In [ ]:
def adjust_brightness(img, factor=1.5):

    # Preserve the alpha channel of RGB images
    if img.ndim == 3 and img.shape[-1] > 3:
        alpha = img[:, :, img.shape[-1] - 1:img.shape[-1]]

    img = factor * img

    if img.ndim == 3 and img.shape[-1] > 3:
        img[:, :, img.shape[-1] - 1:img.shape[-1]] = alpha
        
    img = np.clip(img, a_min=0, a_max=255)

    return img.astype(np.uint8)

#### Contrast adjustment

For contrast adjustment, I provide 2 types of modes: `linear` - a simple equation that multiplies the current pixel value to a factor, and `curve` - which uses a simple power-law S-curve.

In [ ]:
def lower_curve(x, gamma): 
    return 0.5 * (x / 0.5)**gamma


def upper_curve(x, gamma):
    return 1 - 0.5 * ((1 - x) / 0.5)**gamma


def adjust_contrast(img, factor, mode="multiply"):

    # Preserve the alpha channel of RGB images
    if img.ndim == 3 and img.shape[-1] > 3:
        alpha = img[:, :, img.shape[-1] - 1:img.shape[-1]]

    if mode == "curve":
        normalized = img / 255
        normalized[:,:,:][normalized < 0.5] = lower_curve(normalized[:,:,:][normalized < 0.5], factor)
        normalized[:,:,:][normalized >= 0.5] = upper_curve(normalized[:,:,:][normalized >= 0.5], factor)

        normalized = np.clip(normalized, a_min=0, a_max=1)

        processed = normalized * 255

        if img.ndim == 3 and img.shape[-1] > 3:
            processed[:, :, img.shape[-1] - 1:img.shape[-1]] = alpha
        
        return processed.astype(np.uint8)
    
    processed = factor * (img / 255 - 0.5) + 0.5
    processed = processed * 255

    if img.ndim == 3 and img.shape[-1] > 3:
        processed[:, :, img.shape[-1] - 1:img.shape[-1]] = alpha
        
    processed = np.clip(processed, a_min=0, a_max=255)
    
    return processed.astype(np.uint8)

#### Flip horizontally/vertically

In [ ]:
def flip(img, mode):
    # Horizontal flip
    if mode == "1":
        return np.flip(img, axis=1)
    
    # Vertical flip
    else:
        return np.flip(img, axis=0)

#### Convert to grayscale

In [ ]:
def to_grayscale(img):
    if img.ndim != 3:
        raise ValueError("Cannot convert non-RGB to grayscale")

    num_channels = img.shape[-1]

    if num_channels > 3:
        print("WARNING: Alpha channel and will be disregarded. Non-RGB images may result in undesireable outputs")
    elif num_channels < 3:
        raise ValueError("Cannot convert non-RGB to grayscale")

    grayscale = np.dot(img[:, :, :3], [0.299, 0.587, 0.114])
    grayscale = np.clip(grayscale, a_min=0, a_max=255)
    
    return grayscale.astype(np.uint8)

#### Convert to sepia

In [ ]:
def to_sepia(img):
    if img.ndim != 3:
        raise ValueError("Cannot convert non-RGB to sepia")
    
    num_channels = img.shape[-1]

    if num_channels > 3:
        print("WARNING: Alpha channel and will be disregarded. Non-RGB images may result in undesireable outputs")
    elif num_channels < 3:
        raise ValueError("Cannot convert non-RGB to sepia")

    red = img[:, :, :1]
    green = img[:, :, 1:2]
    blue = img[:, :, 2:3]

    sep_red = 0.393 * red + 0.769 * green + 0.189 * blue
    sep_green = 0.349 * red + 0.686 * green + 0.168 * blue
    sep_blue = 0.272 * red + 0.534 * green + 0.131 * blue

    sepia = np.concatenate((sep_red, sep_green, sep_blue), axis=2)
    sepia = np.clip(sepia, a_min=0, a_max=255)

    return sepia.astype(np.uint8)

#### Blur

The details of the blurring algorithm is already covered in the report. In short, the program will generate a 2D Gaussian kernel based on the user's inputs. Then the kernel is convolved with the original image via the sliding window technique.

General 2-dimensional Gaussian function, which is in the shape of a bell curve:

In [ ]:
def gaussian_2d(x, y, sigma):
    coeff = 1. / (2. * np.pi * sigma**2) 
    return coeff * np.exp(
        -1. * ((x**2 + y**2) / (2. * sigma**2))
    ) 

Function to create a kernel with standard deviation `sigma` and the size of `dim`:

In [ ]:
def generate_kernel(sigma, dim=3):
    kernel = np.zeros((dim, dim))
    center = dim // 2

    y, x = np.ogrid[:dim, :dim]
    kernel = gaussian_2d(x - center, y - center, sigma)

    # Normalize so that sum(kernel) = 1
    return kernel / np.sum(kernel)

The `convolve` function will perform the main task of combining our input image with the created kernel:

In [ ]:
def convolve(mat, ker):
    '''
    Convolves a matrix with 2D Gaussian kernel 
    '''

    kdim = ker.shape[0]
    matdim = mat.ndim

    # VERY IMPORTATN!!!!! don't forgor plz
    # Add new axis to allow broadcasting on each kernel element
    if matdim > 2:
        ker = ker[:, :, np.newaxis]

    center = kdim // 2
    
    # Pad the edges of the matrix with 0s
    if matdim > 2:
        padded = np.pad(mat, ((center, center), (center, center), (0, 0)))
    else:
        padded = np.pad(mat, ((center, center), (center, center)))
    
    convolved = np.zeros(mat.shape)

    # Convolve each pixel with the Gaussian kernel
    for r in range(center, padded.shape[0] - center):
        for c in range(center, padded.shape[1] - center):
            pix_vals = padded[r - center:r + 1 + center, c - center:c + 1 + center]

            # Performs the actual convolution on each pixel
            if matdim > 2:
                res = np.sum(np.sum(np.multiply(pix_vals, ker), axis=0), axis=0)
            else:
                res = np.sum(np.multiply(pix_vals, ker))
                
            convolved[r - center][c - center] = res

    return convolved

Finally, the `blur` function will blur the image using the utility functions defined above:

In [ ]:
def blur(img, intensity, size=3):
    if size <= 1:
        size = 3
        
    if size % 2 == 0:
        print("Kernel size must be an odd number, kernel size will be adjusted")
        size += 1

    print("Initiating blurring procedure...")
    start = time.time()

    kernel = generate_kernel(intensity, size)
    print("Kernel generated. Beginning blurring procedure...")

    blurred = convolve(img, kernel)
    end = time.time()
    
    print(f"Finished in {str(end - start)} seconds, convolved with Gaussian 2D kernel:")
    print(f"\tsigma = {str(intensity)}")
    print(f"\tkernel_size = {str(size)}")
    
    return blurred.astype(np.uint8)


#### Sharpen

The sharpening algorithm will make use of the `convolve` function from the previous section. This is the implementation of unsharp masking, which I also mentioned in the report.

In [ ]:
def sharpen(img, amount, size=3):
    '''
    Use unsharp masking to sharpen the image
    '''

    if size <= 1:
        size = 3

    if size % 2 == 0:
        print("Kernel size must be an odd number, kernel size will be adjusted")
        size += 1

    kernel = generate_kernel(sigma=100, dim=size)
    
    print("Kernel generated. Beginning sharpening procedure...")
    start = time.time()
    
    blurred = convolve(img, kernel)

    mask = (img - blurred) * amount
    mask = (mask * 127 / np.max(np.abs(mask))).astype(np.int8)

    sharpened = img + mask
    sharpened = np.clip(sharpened, a_min=0, a_max=255)

    end = time.time()
    print(f"Finished in {str(end - start)} seconds")

    return sharpened.astype(np.uint8)
    

#### Crop from center

In [ ]:
def crop(img, crop_size):
    img_width, img_height = img.shape[1], img.shape[0]
    crop_width, crop_height = crop_size

    if crop_width > img_width or crop_height > img_height:
        raise ValueError("Crop size exceeds image resolution")
    
    start_w, start_h = (img_width - crop_width) // 2, (img_height - crop_height) // 2

    return img[start_h:-start_h, start_w:-start_w]

#### Create a circular mask

In [ ]:
def mask_cir(img):
    applied = img.copy()

    # Grid and center for circular mask
    y, x = np.ogrid[:img.shape[0], :img.shape[1]]
    c_y, c_x = img.shape[0] / 2, img.shape[1] / 2

    radius = min(img.shape[0], img.shape[1]) / 2

    # Matrix of distance between each point and center
    distance = np.sqrt((x - c_x)**2 + (y - c_y)**2)

    cir_mask = distance > radius
    applied[cir_mask] = 0

    return applied

#### Create a cross elliptical mask

In [ ]:
def mask_ell(img):
    applied = img.copy()

    height, width = img.shape[0], img.shape[1]
    angle = np.arctan(height / width)

    major_axis = np.sqrt(height**2 + width**2)
    minor_axis = major_axis / 3
    a = major_axis / 2 
    b = minor_axis / 2 

    c_y, c_x = height // 2, width // 2
    y, x = np.ogrid[:height, :width]
    y, x = y - c_y, x - c_x

    # Ellipse 1
    rotated_x_1 = -1. * np.cos(-1 * angle) * x + np.sin(-1 * angle) * y
    rotated_y_1 = np.sin(-1 * angle) * x + np.cos(-1 * angle) * y
    e_1 = (rotated_x_1)**2 / a**2 + (rotated_y_1)**2 / b**2

    # Ellipse 2
    rotated_x_2 = -1. * np.cos(angle) * x + np.sin(angle) * y
    rotated_y_2 = np.sin(angle) * x + np.cos(angle) * y
    e_2 = (rotated_x_2)**2 / a**2 + (rotated_y_2)**2 / b**2

    ell_mask_1 = e_1 > 1
    ell_mask_2 = e_2 > 1

    applied[ell_mask_1 & ell_mask_2] = 0

    return applied

#### Other utility functions

Although it is unnecessary for this project, I believe having a plot with histograms displaying the distribution of each channel's intensity can help a lot in analyzing images, especially for people who are familiar with media editing softwares like Adobe Lightroom or Photoshop.

In [ ]:
def render_histogram(img, ax):
    if img.ndim == 3:
        num_channels = img.shape[-1]
        img = np.reshape(img, (-1, num_channels))

        if num_channels > 3:
            print("WARNING: Number of channels exceeded 3, alpha channel will not be counted")
            num_channels -= 1

        colors = ["red", "green", "blue"]
        data = np.array([np.take(img, a, axis=1) for a in range(num_channels)])
        for i, d in enumerate(data):
            ax.hist(d, bins=50, range=(0, 255), color=colors[i], alpha=0.3)
            
    else:
        ax.hist(img.flatten(), bins=50, range=(0, 255), color="magenta", alpha=0.3)
        
    ax.set_xlim(0, 255)
    ax.set_xlabel("Channel intensity")
    ax.set_ylabel("Count")


def summary_plot(original, processed, draw_curve=False, gamma=None):
    fig, ax = plt.subplots(2, 2)

    if processed.ndim == 2:
        ax[0][0].imshow(original, cmap="gray")
    else:
        ax[0][0].imshow(original)
        
    render_histogram(original, ax[0][1])
    
    if processed.ndim == 2:
        ax[1][0].imshow(processed, cmap="gray")
    else:
        ax[1][0].imshow(processed)
        
    render_histogram(processed, ax[1][1])

    if draw_curve:
        ax_curve = ax[1][1].inset_axes([.7, .61, .3, .3])
        ax_curve.set_aspect('equal')
        
        x = np.linspace(0, 0.5, 1000)
        lower = lower_curve(x, gamma)
        upper = upper_curve(x + 0.5, gamma)
        ax_curve.plot(x, lower)
        ax_curve.plot(x + 0.5, upper)
        ax_curve.set_title("Tone curve")

    fig.set_size_inches((15, 13), forward=False)
    
    plt.show()

The `handle_input` function is purely to collect information about the image and configuration via user inputs.

In [ ]:
def handle_input():
    modes = {"1": "brightness",
             "2": "contrast",
             "3": "flip",
             "4": "grayscale",
             "5": "sepia",
             "6": "blur",
             "7": "sharpen",
             "8": "crop",
             "9": "cirmask",
             "10": "ellmask",
             "0": "all"}

    img_path = input("Enter image path: ")

    with PIL.Image.open(img_path) as img:
        pixels = np.array(img)

    out_type = input("Enter output file type (png, pdf): ")
    while out_type not in ["png", "pdf"]:
        out_type = input("Unsupported type. Please re-enter: ")

    choice = input("Modes:\n \
                   1 - Adjust brightness \n \
                   2 - Adjust contrast \n \
                   3 - Flip \n \
                   4 - Convert grayscale \n \
                   5 - Convert sepia \n \
                   6 - Blur \n \
                   7 - Sharpen \n \
                   8 - Crop center \n \
                   9 - Apply circular mask \n \
                   10 - Apply elliptical mask \n \
                   0 - All")
    
    while choice not in modes:
        choice = input("Processing mode not found. Please re-enter: ")

    config = []
    
    if choice == "1":
        intensity = input("Enter intensity (> 1 to increase, < 1 to decrease): ")
        intensity = float(intensity)
        config.append(intensity)

    elif choice == "2":
        intensity = input("Enter intensity (> 1 to increase, < 1 to decrease): ")
        intensity = float(intensity)
        config.append(intensity)

        contrast_mode = input("Enter contrast enhancement mode (multiply (default), curve) (leave blank to run as default): ")
        while contrast_mode not in ["", "multiply", "curve"]:
            contrast_mode = input("Invalid mode. Please re-enter: ")

        config.append(contrast_mode)

    elif choice == "3":
        orientation = input("Select flipping mode (1 - horizontal, 2 - vertical): ")
        while orientation not in ["1", "2"]:
            orientation = input("Invalid orientation. Please re-enter: ")

        config.append(orientation)

    elif choice == "6":
        amount = input("Enter amount of blurring: ")
        amount = float(amount)
        config.append(amount)

        radius = input("Enter kernel size (pixels): ")
        radius = int(radius)
        config.append(radius)

    elif choice == "7":
        amount = input("Enter amount of sharpening: ")
        amount = float(amount)
        config.append(amount)

        radius = input("Enter kernel size (pixels): ")
        radius = int(radius)
        config.append(radius)

    elif choice == "8":
        width = input("Enter cropping width: ")
        height = input("Enter cropping height: ")
        crop_size = int(width), int(height)
        config.append(crop_size)

    elif choice == "0":
        # --- Brightness adjustment
        b_intensity = input("Enter brightness intensity (> 1 to increase, < 1 to decrease): ")
        b_intensity = float(b_intensity)
        config.append(b_intensity)

        # --- Contrast adjustment
        contrast = []
        c_intensity = input("Enter contrast intensity (> 1 to increase, < 1 to decrease): ")
        c_intensity = float(c_intensity)
        contrast.append(c_intensity)

        contrast_mode = input("Enter contrast enhancement mode (multiply (default), curve) (leave blank to run as default): ")
        while contrast_mode not in ["", "multiply", "curve"]:
            contrast_mode = input("Invalid mode. Please re-enter: ")

        contrast.append(contrast_mode)
        config.append(contrast)

        # --- Flip
        orientation = input("Select flipping mode (1 - horizontal, 2 - vertical): ")
        while orientation not in ["1", "2"]:
            orientation = input("Invalid orientation. Please re-enter: ")

        config.append(orientation)

        # --- Blurring 
        blur = []
        amount = input("Enter amount of blurring: ")
        amount = float(amount)
        blur.append(amount)

        radius = input("Enter blurring radius (pixels): ")
        radius = int(radius)
        blur.append(radius)

        config.append(blur)

        # --- Sharpening
        sharpen = []
        amount = input("Enter amount of sharpening: ")
        amount = float(amount)
        sharpen.append(amount)

        radius = input("Enter sharpening radius (pixels): ")
        radius = int(radius)
        sharpen.append(radius)

        config.append(sharpen)

        # --- Cropping
        width = input("Enter cropping width: ")
        height = input("Enter cropping height: ")
        crop_size = int(width), int(height)
        config.append(crop_size)


    return img_path, pixels, out_type, modes[choice], config

Finally, the `main` function. All of the pre-defined tasks will be called here, thus executing the entire program.

In [ ]:
def main():
    img_path, img, out_type, mode, config = handle_input()

    img_name = ""
    img_path = img_path.split('.')
    img_name = img_name.join(img_path[:-1])
    # img_name = "output/" + img_name

    original = img.copy()

    draw_curve = False
    gamma = None

    if mode == "brightness":
        processed = adjust_brightness(img, factor=config[0])
        
    elif mode == "contrast":
        intensity, cont_mode = config
        processed = adjust_contrast(img, intensity, mode=cont_mode)
        if cont_mode == "curve":
            draw_curve = True
            gamma = intensity

    elif mode == "flip": processed = flip(img, config[0])
    elif mode == "grayscale": processed = to_grayscale(img)
    elif mode == "sepia": processed = to_sepia(img)

    elif mode == "blur":
        amount, radius = config
        processed = blur(img, amount, size=radius)

    elif mode == "sharpen":
        amount, radius = config
        processed = sharpen(img, amount, size=radius)

    elif mode == "crop": processed = crop(img, crop_size=config[0])
    elif mode == "cirmask": processed = mask_cir(img)
    elif mode == "ellmask": processed = mask_ell(img)

    elif mode == "all":
        config_brightness = config[0]
        config_contrast = config[1]
        config_flip = config[2]
        config_blur = config[3]
        config_sharpen = config[4]
        config_crop = config[5]

        bright = adjust_brightness(img.copy(), factor=config_brightness)
        bright = np.ascontiguousarray(bright)
        plt.imsave(img_name + "_brightness." + out_type, bright)
        print("--- Brightness adjusted")

        contrast = adjust_contrast(img.copy(), config_contrast[0], mode=config_contrast[1])
        contrast = np.ascontiguousarray(contrast)
        plt.imsave(img_name + "_contrast." + out_type, contrast)
        print("--- Contrast adjusted")

        flipped = flip(img.copy(), config_flip)
        flipped = np.ascontiguousarray(flipped)
        plt.imsave(img_name + "_flip." + out_type, flipped)
        print("--- Flipped")

        gray = to_grayscale(img.copy())
        gray = np.ascontiguousarray(gray)
        plt.imsave(img_name + "_grayscale." + out_type, gray, cmap="gray")
        print("--- Converted to grayscale")

        sepia = to_sepia(img.copy())
        sepia = np.ascontiguousarray(sepia)
        plt.imsave(img_name + "_sepia." + out_type, sepia)
        print("--- Converted to sepia")

        blurred = blur(img.copy(), config_blur[0], config_blur[1])
        blurred = np.ascontiguousarray(blurred)
        plt.imsave(img_name + "_blur." + out_type, blurred)
        print("--- Blurred")

        sharpened = sharpen(img.copy(), config_sharpen[0], config_sharpen[1])
        sharpened = np.ascontiguousarray(sharpened)
        plt.imsave(img_name + "_sharpen." + out_type, sharpened)
        print("--- Sharpened")

        cropped = crop(img.copy(), config_crop)
        cropped = np.ascontiguousarray(cropped)
        plt.imsave(img_name + "_crop." + out_type, cropped)
        print("--- Cropped")

        masked_cir = mask_cir(img.copy())
        masked_cir = np.ascontiguousarray(masked_cir)
        plt.imsave(img_name + "_circular_mask." + out_type, masked_cir)
        print("--- Applied circular mask")
        
        masked_ell = mask_ell(img.copy())
        masked_ell = np.ascontiguousarray(masked_ell)
        plt.imsave(img_name + "_elliptical_mask." + out_type, masked_ell)
        print("--- Applied elliptical mask")

    if mode != "all":
        processed = np.ascontiguousarray(processed)

        if mode == "grayscale":
            plt.imsave(img_name + "_" + mode + "." + out_type, processed, cmap="gray")
        else:
            plt.imsave(img_name + "_" + mode + "." + out_type, processed)

        # Plots original and processed, with histogram 
        summary_plot(original, processed, draw_curve=draw_curve, gamma=gamma)

Let us run the `main` program and observe the results!

In [ ]:
main()

### REFERENCES
Please refer to the PDF file to view full citations.